In [2]:
## Preamble: Package Loading
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter

<h4> Block Diagonal Matrix Function </h4>

In [224]:
def blkdiag(mat,nb):
    """
INPUTS
mat     Square Matrix which will form the block in a block diagonal matrix
nb      Number of diagonal block in output matrix 

OUTPUT
v       Block diagonal matrix of dimension ( nb*mat.shape[0] x nb*mat.shape[0] )
    """
    # Initializing the varcov matrix for all crosssections
    v = np.hstack((mat,np.zeros((mat.shape[0],(nb-1)*(mat.shape[1])))))
    # Registry matrix used in following loops
    vreg = np.eye(nb-1)
    for j in np.arange(nb-1):
        # Initializing current block of rows 
        pv = np.zeros((mat.shape[0],mat.shape[0]))
        # Horiz Stacking either zeros or var_err depending on ve_reg[j,i]
        for i in np.arange(nb-1):
            if vreg[j,i] == 1: # Stack var_err onto pv_err
                pv = np.hstack((pv,mat))
            if vreg[j,i] == 0: # Stack zeros onto pv_err
                pv = np.hstack((pv,np.zeros((mat.shape[0],mat.shape[0]))))
        # Vertically stacking block rows on top of one another
        v = np.vstack((v,pv))
    return v

#### Block Diagonal Matrix Function Demonstration

In [226]:
blkdiag(np.ones((3,3)),4)

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

<h3> DGP Inputs </h3>

In [186]:

# Setting a Seed
np.random.seed([10])

# Number of Time Periods
ntp = 10
# Number of Cross-Sections
ncs = 3
# Number of Endogenous Variables in Primary Equation
n_end = 2
# Number of Endogenous Variables in Primary Equation
n_exo = 2
# Total Number of Instruments
t_inst = 10
# Number of Instruments per Crossection
c_inst = 3
# Indicator for whether to force additive non linear cntrl function. 1 = yes 
frc = 0

# Vector of exog off diagonal covariances i.e. cov(Z2t_l,Z2t_(l+j)) = ex_vpro[j-1]
ex_vpro = [0.5]
# Vector of inst off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
inst_vpro = [0.8 , 0.6 , 0.4, 0.2]
# Vector of error off diagonal covariances i.e. cov(Wt_l,Wt_(l+j)) = inst_vpro[j-1]
err_vpro = [0.8 , 0.6 ]

<h2> DGP Generation Code </h2>

In [235]:

# Collection of all Coefficients on Instruments 
icoeffs_reg = list(iter.product([-1,1],repeat = c_inst))
# Collection of all Coefficients on Exogenous Variables
excoeffs_reg = list(iter.product([-1,1],repeat = n_exo))
# Registry of instrument assignments
insts_reg = list(iter.combinations(np.arange(1,t_inst+1),c_inst))
# Iterator for instrument coefficients
lpw_iter = [(x,y) for y in np.arange(1,len(coeff)+1) for x in np.arange(1,len(insts)+1)]
# Iterator for exogenous variables coefficients
lpz_iter = []
for x in np.arange(1,np.ceil(ncs/4)+1): 
    for y in [0,1,2,3] : lpz_iter.append(y)

# Initializing the full coefficient matrix
coeffs = np.zeros((ncs,n_exo+t_inst))

# Generating coefficients matrix
for i in np.arange(ncs): 
    # List of which instruments (col #'s) are used for crossection i
    cr = insts_reg[np.random.randint(len(insts_reg))]
    # List of coefficients which correspond to those instruments
    cfs = icoeffs_reg[np.random.randint(len(icoeffs_reg))]
    for j in np.arange(n_exo):
        # Setting the coeff for each exogenous regression
        coeffs[i,j] = excoeffs_reg[lpz_iter[i]][j]
    for j in np.arange(c_nist):
        # Setting the coeff for each instrument used by i
        coeffs[i,n_exo-1+cr[j]] = cfs[j]
            
## Joint Distribution of Exogenous regressors
# Vector of Means (=0)
mu_ex = np.zeros(n_exo)
# Diagonal matrix of variaces (=1)
var_ex = np.eye(n_exo)
# Variance Covariance Matrix Generation or EACH crossection
for i in np.arange(len(ex_vpro)):
    var_ex = (var_ex + ex_vpro[i]*np.eye(n_exo,k=i+1)
                         + ex_vpro[i]*np.eye(n_exo,k=-(i+1))) 
# Exogenous regressor mean vector for ALL Crossections
Mu_ex = np.tile(mu_ex,ncs) 
# Exogenous regressor variance covariance matrix for ALL Crossections
V_ex = blkdiag(var_ex,ncs)
    
## Joint Distribution of Instruments for all cross sections
# Vector of Means (=0)
mu_inst = np.zeros(c_inst)
# Diagonal Matrix of Variaces (=1)
var_inst = np.eye(t_inst)
# Variance Covariance Matrix Generation
for i in np.arange(len(inst_vpro)):
    var_inst = (var_inst + inst_vpro[i]*np.eye(t_inst,k=i+1) 
                         + inst_vpro[i]*np.eye(t_inst,k=-(i+1)))
    
## Joint Distribution of Error Terms for EACH crossection
# Vector of means
mu_err = np.zeros(n_end+1)
# Diagonal Matrix of Variances 
var_err = np.eye(n_end+1)
# Variance Covariance Matrix Generation
if frc == 0 : 
    # Var Cov matrix for correlated errors ==> additive linear control functions 
    for i in np.arange(len(err_vpro)):
        var_err = (var_err + err_vpro[i]*np.eye(n_end+1,k=i+1) 
                             + err_vpro[i]*np.eye(n_end+1,k=-(i+1)))
# Error term mean vector for ALL Crossections
Mu_err = np.tile(mu_err,ncs)        
# Error term variance covariance matrix for ALL Crossections
V_err = blkdiag(var_err,ncs)

# Exogenous Regressor Generation
Ex = np.random.multivariate_normal(Mu_ex,V_ex,ntp)

# 
Inst = np.random.multivariate_normal(mu_inst,var_inst,ntp)

#
Err = np.random.multivariate_normal(Mu_err,V_err,ntp)



In [236]:
coeffs

array([[-1., -1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [-1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 1., -1.,  1.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.]])

In [237]:
var_inst

array([[ 1. ,  0.8,  0.6,  0.4,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.8,  1. ,  0.8,  0.6,  0.4,  0.2,  0. ,  0. ,  0. ,  0. ],
       [ 0.6,  0.8,  1. ,  0.8,  0.6,  0.4,  0.2,  0. ,  0. ,  0. ],
       [ 0.4,  0.6,  0.8,  1. ,  0.8,  0.6,  0.4,  0.2,  0. ,  0. ],
       [ 0.2,  0.4,  0.6,  0.8,  1. ,  0.8,  0.6,  0.4,  0.2,  0. ],
       [ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  0.8,  0.6,  0.4,  0.2],
       [ 0. ,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  0.8,  0.6,  0.4],
       [ 0. ,  0. ,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  0.8,  0.6],
       [ 0. ,  0. ,  0. ,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  0.8],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ]])

In [238]:
V_ex

array([[ 1. ,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  1. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  1. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  1. ]])

In [239]:
V_err

array([[ 1. ,  0.8,  0.6,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.8,  1. ,  0.8,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.6,  0.8,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0.8,  0.6,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.8,  1. ,  0.8,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.6,  0.8,  1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.8,  0.6],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.8,  1. ,  0.8],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.6,  0.8,  1. ]])